In [2]:
import pandas as pd
import numpy as np

In [3]:
with open("MTA_Subway_Hourly_Ridership__Beginning_February_2022.csv","r") as f:
    df = pd.read_csv(f)
df.head()

/var/folders/wc/9wclntk940q3y52dpvt1g_s80000gn/T/ipykernel_63952/3920661926.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f)


,transit_timestamp,station_complex_id,station_complex,borough,routes,payment_method,ridership,transfers,latitude,longitude,Georeference,itsuid
0,10/14/2022 05:00:00 AM,H007,1 Av (L),M,L,omny,31,0,40.730953,-73.981628,POINT (-73.98162841796875 40.730953216552734),2022-10-14T05:00:00H0071 Av (L)MLomny
1,11/05/2022 03:00:00 AM,H007,1 Av (L),M,L,omny,78,0,40.730953,-73.981628,POINT (-73.98162841796875 40.730953216552734),2022-11-05T03:00:00H0071 Av (L)MLomny
2,01/14/2023 03:00:00 PM,H007,1 Av (L),M,L,omny,14,0,40.730953,-73.981628,POINT (-73.98162841796875 40.730953216552734),2023-01-14T15:00:00H0071 Av (L)MLomny
3,03/23/2023 03:00:00 AM,H007,1 Av (L),M,L,omny,10,0,40.730953,-73.981628,POINT (-73.98162841796875 40.730953216552734),2023-03-23T03:00:00H0071 Av (L)MLomny
4,06/24/2023 06:00:00 AM,H007,1 Av (L),M,L,omny,44,0,40.730953,-73.981628,POINT (-73.98162841796875 40.730953216552734),2023-06-24T06:00:00H0071 Av (L)MLomny


In [11]:
subset_date = "12/25/2022"
subset = df[df["transit_timestamp"].str.contains(subset_date)].reset_index(drop=True)
subset = subset.drop(columns = ["transit_timestamp","Georeference","itsuid","station_complex"])
subset.head()

,station_complex_id,borough,routes,payment_method,ridership,transfers,latitude,longitude
0,R170,M,1,omny,59,0,40.799446,-73.968376
1,H019,BK,L,omny,20,0,40.706154,-73.933144
2,H007,M,L,omny,72,0,40.730953,-73.981628
3,H007,M,L,omny,8,0,40.730953,-73.981628
4,H007,M,L,omny,131,7,40.730953,-73.981628


In [13]:
subset["ridership"] = pd.to_numeric(subset["ridership"])
subset["transfers"] = pd.to_numeric(subset["transfers"])
subset["latitude"] = pd.to_numeric(subset["latitude"])
subset["longitude"] = pd.to_numeric(subset["longitude"])
subset = subset.groupby([c for c in subset.columns if c not in ["ridership","transfers"]]).agg({c:"sum" for c in ["ridership","transfers"]}).reset_index()

In [14]:
subset["routes"] = subset["routes"].astype(str)
subset["routes"] = subset["routes"].apply(lambda x:  [item.strip() for item in x.split(',') if item.strip()])

In [15]:
subset.head()

,station_complex_id,borough,routes,payment_method,latitude,longitude,ridership,transfers
0,A002,M,"[5, N, R, 4, W, 6]",metrocard,40.762661,-73.967255,4557,333
1,A002,M,"[5, N, R, 4, W, 6]",omny,40.762661,-73.967255,2695,396
2,A006,M,"[N, R, W]",metrocard,40.764812,-73.973351,3462,24
3,A006,M,"[N, R, W]",omny,40.764812,-73.973351,2621,38
4,A010,M,"[N, R, Q, W]",metrocard,40.764664,-73.980659,4661,41


In [16]:
med_lat = np.median(list(subset["latitude"].unique()))
med_long = np.median(list(subset["longitude"].unique()))

def assign_region(long, lat):
    region = "S" if lat <= med_lat else "N"
    region += "W" if long <= med_long else "E"
    return region

subset["region"] = subset.apply(lambda row: assign_region(row["longitude"], row["latitude"]), axis=1)
subset.head()

,station_complex_id,borough,routes,payment_method,latitude,longitude,ridership,transfers,region
0,A002,M,"[5, N, R, 4, W, 6]",metrocard,40.762661,-73.967255,4557,333,NW
1,A002,M,"[5, N, R, 4, W, 6]",omny,40.762661,-73.967255,2695,396,NW
2,A006,M,"[N, R, W]",metrocard,40.764812,-73.973351,3462,24,NW
3,A006,M,"[N, R, W]",omny,40.764812,-73.973351,2621,38,NW
4,A010,M,"[N, R, Q, W]",metrocard,40.764664,-73.980659,4661,41,NW


In [17]:
with open("cleaned.csv", "w+") as o:
    subset.to_csv(o, index=False)